In [2]:
from pathlib import Path

import networkx as nx
import pandas as pd

from networkx.algorithms.traversal import bfs_tree

"""
Load data from file
"""

# set up some paths for ins and outs
base_directory = Path.cwd().parent.absolute()
source_data_path = base_directory / "data" / "raw" / "etymwn.tsv"
processed_directory = base_directory / "data" / "processed"
output_graph_yaml_file = processed_directory / "graph_output.yaml"
output_graph_dot_file = processed_directory / "graph_output.dot"
output_graph_png = processed_directory / "graph_output.png"

# load from CSV
cleaned_df = pd.read_csv(
    source_data_path, sep="\t", names=["source_node", "edge_type", "target_node"]
)



# Cleaning notes

## Relationship types found in source data

Relationships are recorded bidirectionally. So a root word will link to its derivatives, and each derivative will link back to the root. To simplify the graph,
I will drop edges that point from derivatives to roots, since that information is already encoded in the root-to-leaf edge and networkx can handle bidirectional
traversal without requiring multiple edges to link the same pair of nodes.

Below are the types of relationships extracted from the data, with comments indicating their directionality.


<- A is the source of B

-> B is the source of A


- "rel:etymology"               ->
- "rel:etymological_origin_of"  <-
- "rel:is_derived_from"         ->
- "rel:has_derived_form"        <-
- "rel:etymologically_related"  <->
- "rel:variant:orthography"     <->

source data also includes a handful of malformed values, which should be dropped or replaced 
- "rel:etymologically" -> "rel:etymologically_related"
- "rel:derived" -> "rel:is_derived_from"



In [3]:

"""
Clean data
"""

# filter out bidirectional relationships and select one directionality to normalize the graph
# I would normally clean to fix a handful of malformed tags as below but we are dropping those edge types anyway
# so instead we will stick to the edge types that point from root words to derived words
root_first_rel_types = ["rel:etymological_origin_of", "rel:has_derived_form"]
cleaned_df = cleaned_df.loc[(cleaned_df["edge_type"].isin(root_first_rel_types))]
cleaned_df[["source_language", "source_word"]] = cleaned_df.source_node.str.split(
    ": ", expand=True
)

# there are a handful of nodes that include strange characters or a :Category: tag that introduces a third
# column for no reason. This data is uninteresting so we can just ignore it and no include it in the graph when we construct it
cleaned_df[["target_language", "target_word", "crud"]] =  cleaned_df.target_node.str.split(": ", expand=True)


In [12]:

"""
Get list of unique languages in data set.

Not super necessary but helpful for understanding the likely subgraph structure. I would guess that the individual
languages will be highly connected/clustered. I suspect the boundaries will blur a bit around the proto- and ancient languages, particularly for languages with
many ancestors in the data set, e.g., Latin
"""

unique_languages = set(cleaned_df["source_language"].unique()).union(set(cleaned_df["target_language"].unique()))
print(sorted(unique_languages))


{'ave', 'frk', 'xno', 'hif', 'sot', 'sun', 'cic', 'jbo', 'umu', 'twf', 'ara', 'tam', 'adt', 'ast', 'mar', 'lij', 'grn', 'kri', 'auc', 'chc', 'kan', 'heb', 'slv', 'chr', 'kjh', 'urd', 'ukr', 'tha', 'szl', 'lim', 'mod', 'p_gmw', 'frp', 'lkt', 'gla', 'tew', 'tat', 'xon', 'rom', 'hye', 'chu', 'tuk', 'sco', 'slk', 'syc', 'haw', 'efi', 'p_sla', 'nor', 'aaq', 'dan', 'ppl', 'wit', 'akk', 'xng', 'txb', 'lmo', 'zko', 'gsw', 'tsn', 'aii', 'hsb', 'nap', 'arw', 'vls', 'byn', 'san', 'kur', 'swa', 'nob', 'pro', 'rap', 'tgl', 'ota', 'gwi', 'cym', 'cha', 'sme', 'nld', 'wln', 'nan', 'dsb', 'zai', 'gmy', 'ksd', 'tnq', 'tir', 'ron', 'sms', 'ike', 'mri', 'dtd', 'eus', 'som', 'odt', 'oji', 'fry', 'rme', 'stg', 'ben', 'sqi', 'ryu', 'sux', 'sei', 'min', 'epo', 'quc', 'nys', 'see', 'xho', 'mxi', 'prg', 'ksh', 'xaa', 'lua', 'frm', 'mya', 'glv', 'cor', 'osp', 'mal', 'nah', 'kaw', 'ccc', 'hak', 'sat', 'bod', 'ase', 'ale', 'ltc', 'fro', 'gez', 'por', 'mic', 'ood', 'rue', 'abs', 'kmb', 'mfr', 'vai', 'oge', 'gil', '

In [4]:

"""
Construct networkx graph
"""

# start with directed so we can preserve directionality data, retaining the option to convert to undirected later to use networkx undirected algorithms
graph = nx.from_pandas_edgelist(
    cleaned_df,
    edge_attr=[
        "edge_type",
        "source_language",
        "source_word",
        "target_language",
        "target_word",
    ],
    source="source_node",
    target="target_node",
    create_using=nx.DiGraph,
)
print(nx.info(graph))


Name: 
Type: DiGraph
Number of nodes: 2743118
Number of edges: 2692096
Average in degree:   0.9814
Average out degree:   0.9814



# What next?

Now that we've loaded the data into a graph, it's time to do some actual analysis. But to do so, we need to define the problem more clearly. Otherwise, there is
an intractable amount of data for many graph algorithms. Pruning to relevant subgraphs would be desirable as an initial post-processing step.

I am primarily interested in English language entries. However, ~~many~~ all of the English words are derived from non-English words. It would be good to prune
entries that are not etymological roots of English words.

My initial instinct is to trim any descendant nodes of English words that are in other languages. Then, we can run BFS from each English node to gather its
ancestors, knowing the descendant nodes have already been trimmed. This could accidentally exclude relevant data if there are derivation paths that jump from
English to another language and then back, but that's an interesting question in and of itself and might be worth investigating as preliminary matter before
pursuing this approach. 

However, I suspect the BFS approach may be extremely inefficient and that it will be necessary to reduce the size of the search space to something more
tractable. 

So. What exactly are we looking for?

- Nodes with the "eng:" prefix
- Nodes that are direct and indirect ancestors of the English nodes



In [ ]:
"""
DAG analysis

Check if graph is directed acyclic
"""

from networkx.algorithms.dag import is_directed_acyclic_graph
is_dag = is_directed_acyclic_graph(graph)
print(is_dag)


In [10]:
"""
DAG analysis

I expected that the graph would be acyclic, because logically it doesn't make sense of a word to be an ancestor of a word that is also the first word's ancestor.
However, is_directed_acyclic_graph returned false, so I am going to check for cycles and see if there's something in the source data that can be cleaned up to
enable DAG analysis
"""
from networkx.algorithms.cycles import simple_cycles
cycles = list(simple_cycles(graph))
cycle_count = len(cycles)
print(cycle_count)


3305


In [18]:
"""
DAG analysis

So... looks like all the cycles are self-connected single nodes. I'm guessing if I prune those it will be acyclic.

Tried this, and still there were some loops left. they were in somewhat obscure languages so I decided to just drop them all.
graph.remove_edges_from(nx.selfloop_edges(graph))

"""
from networkx.algorithms.dag import is_directed_acyclic_graph
from networkx.algorithms.cycles import simple_cycles
cycle_nodes = [node for cycle in list(simple_cycles(graph)) for node in cycle]
graph.remove_nodes_from(cycle_nodes)
is_dag = is_directed_acyclic_graph(graph)
print(is_dag)


True
0


In [19]:
"""
DAG analysis

Ok we have a DAG, time to see what kinds of fun patterns we can find...

"""
from networkx.algorithms.dag import dag_longest_path 

longest_path = dag_longest_path(graph)
print(longest_path)


['grc: χάλιξ', 'lat: calx', 'lat: calceus', 'lat: calceō', 'lat: calceare', 'ita: calzare', 'ita: calza', 'ita: calzone', 'fra: caleçon', 'fra: caleçons', 'epo: kalsono', 'epo: kalsoneto']


In [20]:
"""
DAG analysis

Create English-related subgraph

Let's see if we can exploit the DAG properties to filter to chains with English words only
"""
roots = [node for node in nx.nodes(graph) if len(nx.ancestors(graph, node)) == 0]
print(len(roots))
english_nodes = [nx.descendants(graph, root) for root in roots if list(filter(lambda x: x[0:3] == "eng", nx.descendants(graph, root)))]
print(len(english_nodes))
english_graph = graph.subgraph([node for desc in english_nodes for node in desc])
print(nx.info(english_graph))


342736
96374
Name: 
Type: DiGraph
Number of nodes: 912653
Number of edges: 715717
Average in degree:   0.7842
Average out degree:   0.7842


In [21]:
"""
English graph analysis

Inspect highest degree nodes
"""
nodes_by_degree = sorted(english_graph.degree, key=lambda x: x[1], reverse=True)
print(nodes_by_degree[0:99])


[('eng: non-', 5748), ('eng: -ly', 5439), ('eng: un-', 4206), ('eng: -like', 2438), ('eng: -er', 2346), ('eng: -less', 1974), ('ita: -mente', 1666), ('eng: -able', 1428), ('eng: time', 1322), ('eng: anti-', 1293), ('eng: -y', 1227), ('eng: poly-', 1004), ('ita: -abile', 880), ('eng: -ic', 812), ('eng: -ed', 754), ('eng: multi-', 715), ('eng: over-', 710), ('spa: -mente', 681), ('fra: -ure', 665), ('eng: -ian', 625), ('eng: -ish', 588), ('eng: -ally', 520), ('eng: -ize', 500), ('eng: dog', 496), ('eng: disease', 493), ('deu: Stein', 485), ('eng: inter-', 461), ('fra: -er', 451), ('eng: hyper-', 445), ('eng: bird', 444), ('fra: -ique', 442), ('ita: -oso', 441), ('eng: be-', 428), ('eng: in-', 426), ('fra: -age', 424), ('ita: -tura', 419), ('eng: micro-', 401), ('eng: de-', 397), ('eng: post-', 395), ('eng: -an', 392), ('eng: -ity', 383), ('eng: -ing', 383), ('eng: -ship', 380), ('gle: -acht', 376), ('ita: -mento', 356), ('eng: Chinese', 346), ('eng: semi-', 327), ('eng: back', 326), ('it

In [7]:
"""
Undirected graph analysis

Construct undirected graph
"""

# convert to undirected so we can apply undirected algorithms
undirected_graph = graph.to_undirected()
print(nx.info(undirected_graph))


KeyboardInterrupt: 

In [17]:
"""
Undirected graph analysis

Check for connected components
"""

from networkx.algorithms.components import connected_components

# check for connected components. may be a way to prune subgraphs that do not relate to English etymology
conn_components = list(connected_components(undirected_graph))
connected_component_count = len(conn_components)
print(connected_component_count)


209375


In [ ]:

"""
"""

# grab the nodes that have the eng tag, then build the connected graph from those
english_nodes = [n for n in graph.nodes() if n[0:3] == 'eng']
nodes_to_add = []
# add nodes inside loop to avoid having to flatten later
[nodes_to_add.extend(bfs_tree(graph, source=node)) for node in english_nodes]

english_graph = graph.subgraph(nodes_to_add)
print(nx.info(english_graph))


In [18]:

nodes_by_degree = sorted(english_graph.degree, key=lambda x: x[1], reverse=True)
print(nodes_by_degree[0:99])


NameError: name 'english_graph' is not defined